In [2]:
from transformers import AutoModel, AutoTokenizer
from transformers import GenerationConfig
import torch
from mtkresearch.llm.prompt import MRPromptV3
import pprint

# model_params_size: either '3B' or '8B'
model_params_size = '3B'
model_id = f"MediaTek-Research/Llama-Breeze2-{model_params_size}-Instruct-v0_1"
model = AutoModel.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map='auto',
    img_context_token_id=128212
).eval()

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, use_fast=False)

generation_config = GenerationConfig(
  max_new_tokens=2048,
  do_sample=True,
  temperature=0.01,
  top_p=0.01,
  repetition_penalty=1.1,
  eos_token_id=128009
)

prompt_engine = MRPromptV3()

sys_prompt = 'You are a helpful AI assistant built by MediaTek Research. The user you are helping speaks Traditional Chinese and comes from Taiwan.'

def _inference(tokenizer, model, generation_config, prompt, pixel_values=None):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    if pixel_values is None:
        output_tensors = model.generate(**inputs, generation_config=generation_config)
    else:
        output_tensors = model.generate(**inputs, generation_config=generation_config, pixel_values=pixel_values.to(model.dtype))
    output_str = tokenizer.decode(output_tensors[0])
    return output_str

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
conversations = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": "請幫我介紹珍珠奶茶"},
]

prompt = prompt_engine.get_prompt(conversations)
output_str = _inference(tokenizer, model, generation_config, prompt)
result = prompt_engine.parse_generated_str(output_str)
pprint.pp(result)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'assistant',
 'content': '珍珠奶茶，又稱為珍珠奶綠，是一種流行於臺灣的飲料。它由紅茶、牛奶或豆漿以及加入黑糖熬煮而成的珍珠組合而成。\n'
            '\n'
            '1. 材料：\n'
            '- 紅茶\n'
            '- 牛奶（可選）\n'
            '- 豆漿（可選）\n'
            '- 黑糖水\n'
            '- 珍珠\n'
            '\n'
            '2. 做法：\n'
            'a. 準備一個鍋子，將黑糖水加熱至沸騰。\n'
            'b. 在另一個碗中混合紅茶和牛奶/豆漿。\n'
            'c. 將沸騰的黑糖水倒入紅茶中攪拌均勻。\n'
            'd. 如果需要，可以根據個人喜好添加冰塊。\n'
            'e. 最後，將珍珠放入並攪拌均勻即可享用。\n'
            '\n'
            '3. 飲用方式：\n'
            '- 用吸管喝珍珠奶茶是最常見的方法。\n'
            '- 也可以使用湯匙來吃珍珠。\n'
            '\n'
            '4. 注意事項：\n'
            '- 適量食用，因為珍珠含有大量的卡路里。\n'
            '- 如果不習慣甜味，可以減少黑糖水的份量。\n'
            '\n'
            '5. 變化：\n'
            '- 可以嘗試不同的口味，如抹茶、巧克力或草莓等。\n'
            '- 也可以在紅茶中加入其他配料，如椰果或粉圓。\n'
            '\n'
            '6. 傳統與現代：\n'
            '- 珍珠奶茶起源於台灣夜市文化，並隨著時間發展成為全球受歡迎的飲品。\n'
            '- 現在，許多餐廳提供各種不同風格的珍珠奶茶，包括手工製作的珍珠和獨特的配料。\n'
            '\n'
            '總